In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit


# -----------------------------
# 1) CME 전처리: 월별 txt 모두 로드 -> Halo만 -> 일별 합/횟수 -> 누락일 0 채움
# -----------------------------
import pandas as pd
import numpy as np
from pathlib import Path

def load_and_preprocess_cme_multifeature(year: int, cme_dir: str):

    cme_dir = Path(cme_dir)
    files = [cme_dir / f"univ{year}_{m:02d}.txt" for m in range(1, 13)]

    dfs = []
    for f in files:
        if not f.exists():
            continue

        # fixed width
        df = pd.read_fwf(
            f,
            skiprows=4,
            header=None
        )
        dfs.append(df)

    cme = pd.concat(dfs, ignore_index=True)

    # -----------------------------------
    # 컬럼 구조 (실제 카탈로그 기준)
    # -----------------------------------
    # 0 : Date
    # 1 : Time
    # 2 : Central PA   (or Halo)
    # 3 : Width
    # 4 : Linear speed
    # 5 : 2nd order speed (initial)
    # 6 : 2nd order speed (final)
    # 7 : 2nd order speed (20R)
    # 8 : Accel
    # 9 : Mass
    # 10: Kinetic energy
    # 11: MPA
    # 12: Remarks

    out = pd.DataFrame()

    out["date"] = pd.to_datetime(cme.iloc[:, 0], errors="coerce")

    # halo / partial halo flag
    central = cme.iloc[:, 2].astype(str)
    out["is_halo"] = central.str.contains("Halo", case=False, na=False).astype(int)
    out["is_partial_halo"] = cme.iloc[:, 12].astype(str).str.contains(
        "Partial", case=False, na=False
    ).astype(int)

    # numeric columns
    def num(col):
        return pd.to_numeric(
            cme.iloc[:, col].astype(str).str.replace("*", "", regex=False),
            errors="coerce"
        )

    out["width"] = num(3)
    out["speed_linear"] = num(4)
    out["speed_init"] = num(5)
    out["speed_final"] = num(6)
    out["speed_20R"] = num(7)
    out["accel"] = num(8)

    # mass, energy는 ------- 같은 값이 있으므로 그대로 numeric 처리
    out["mass"] = num(9)
    out["kinetic_energy"] = num(10)

    out = out.dropna(subset=["date"])

    # -----------------------------------
    # 하루 단위 aggregation
    # -----------------------------------
    daily = (
        out
        .groupby("date")
        .agg(
            cme_count = ("date", "count"),

            halo_count = ("is_halo", "sum"),
            partial_halo_count = ("is_partial_halo", "sum"),

            width_max = ("width", "max"),
            width_mean = ("width", "mean"),

            speed_linear_max = ("speed_linear", "max"),
            speed_linear_mean = ("speed_linear", "mean"),

            speed_init_max = ("speed_init", "max"),
            speed_final_max = ("speed_final", "max"),
            speed_20R_max = ("speed_20R", "max"),

            accel_max = ("accel", "max"),
            accel_min = ("accel", "min"),

            mass_sum = ("mass", "sum"),
            mass_max = ("mass", "max"),

            ke_sum = ("kinetic_energy", "sum"),
            ke_max = ("kinetic_energy", "max"),
        )
        .reset_index()
    )

    # -----------------------------------
    # 날짜 채우기
    # -----------------------------------
    full_dates = pd.date_range(
        pd.Timestamp(year, 1, 1),
        pd.Timestamp(year, 12, 31),
        freq="D"
    )

    daily = (
        daily
        .set_index("date")
        .reindex(full_dates)
        .rename_axis("date")
        .reset_index()
    )

    # count 계열은 0, 나머지는 0으로 채워줌(모델용)
    count_cols = [
        "cme_count", "halo_count", "partial_halo_count"
    ]
    for c in count_cols:
        daily[c] = daily[c].fillna(0)

    other_cols = [c for c in daily.columns if c not in ["date"] + count_cols]
    daily[other_cols] = daily[other_cols].fillna(0)

    return daily



# -----------------------------
# 2) hproton 전처리: 메타라인 섞여도 안전하게 -> date 만들기 -> p_gt_100만
# -----------------------------
import pandas as pd
import numpy as np

def load_and_preprocess_hproton(dpd_path: str) -> pd.DataFrame:
    rows = []
    with open(dpd_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            parts = line.split()
            if len(parts) < 6:   # 최소 year month day p1 p10 p100 있어야 함
                continue

            # 첫 3개가 정수여야 데이터 줄로 인정
            try:
                y = int(float(parts[0]))
                m = int(float(parts[1]))
                d = int(float(parts[2]))
            except ValueError:
                continue

            # 데이터 줄이면 앞 9개까지 안전하게 읽기 (없으면 NaN 채움)
            vals = parts[:9] + [np.nan] * (9 - len(parts[:9]))
            rows.append([y, m, d] + vals[3:9])

    hp = pd.DataFrame(
        rows,
        columns=[
            "year", "month", "day",
            "p_gt_1MeV", "p_gt_10MeV", "p_gt_100MeV",
            "e_gt_0p6MeV", "e_gt_2MeV",
            "neutron_pct"
        ]
    )

    # 숫자 변환
    for c in ["p_gt_1MeV","p_gt_10MeV","p_gt_100MeV","e_gt_0p6MeV","e_gt_2MeV","neutron_pct"]:
        hp[c] = pd.to_numeric(hp[c], errors="coerce")

    hp["date"] = pd.to_datetime(hp[["year","month","day"]])
    hp = hp[["date", "p_gt_100MeV"]].sort_values("date").reset_index(drop=True)

    # 결측은 0으로(원하면 그대로 NaN 둬도 됨)
    hp["p_gt_100MeV"] = hp["p_gt_100MeV"].fillna(0.0)

    return hp



# -----------------------------
# 3) CME + hproton 길이 n 맞추기(동일 date index로 reindex)
# -----------------------------
def align_daily_series(cme_daily: pd.DataFrame, hp_daily: pd.DataFrame) -> pd.DataFrame:
    # 공통 날짜 범위(교집합)로 맞추고 싶으면 여기서 start/end 조절 가능
    start = max(cme_daily["date"].min(), hp_daily["date"].min())
    end   = min(cme_daily["date"].max(), hp_daily["date"].max())
    full_dates = pd.date_range(start, end, freq="D")

    cme_aligned = (
        cme_daily.set_index("date")
        .reindex(full_dates, fill_value=0)
        .rename_axis("date")
        .reset_index()
    )
    hp_aligned = (
        hp_daily.set_index("date")
        .reindex(full_dates, fill_value=0)
        .rename_axis("date")
        .reset_index()
    )

    merged = pd.merge(cme_aligned, hp_aligned, on="date", how="inner")
    return merged


# -----------------------------
# 4) (과거 k일 CME) -> (오늘 SEP) 학습용 테이블 생성
# -----------------------------
def make_supervised_dataset(
    df: pd.DataFrame,
    base_feature_cols=None,
    k_lag: int = 3,
    future_window: int = 3,
    use_log_target: bool = True
):
    """
    df: must include columns ['date', 'p_gt_100MeV'] + CME feature columns

    base_feature_cols: CME feature columns list.
      - None이면 자동으로 (date, p_gt_100MeV) 제외한 모든 컬럼을 feature로 사용.
    """

    out = df.sort_values("date").reset_index(drop=True).copy()

    # ✅ feature 컬럼 자동 선택
    if base_feature_cols is None:
        base_feature_cols = [c for c in out.columns if c not in ["date", "p_gt_100MeV"]]

    # ------------------------
    # 1) lag / rolling features
    # ------------------------
    feat_cols = []
    for col in base_feature_cols:
        # lag
        for lag in range(1, k_lag + 1):
            name = f"{col}_lag{lag}"
            out[name] = out[col].shift(lag)
            feat_cols.append(name)

        # rolling sum (최근 k일 합)
        name = f"{col}_rollsum{k_lag}"
        out[name] = out[col].shift(1).rolling(k_lag).sum()
        feat_cols.append(name)

        # rolling max (최근 k일 최대)
        name = f"{col}_rollmax{k_lag}"
        out[name] = out[col].shift(1).rolling(k_lag).max()
        feat_cols.append(name)

    # ------------------------
    # 2) target: 미래 window max SEP
    # ------------------------
    out["target_sep"] = (
        out["p_gt_100MeV"]
        .shift(-1)
        .rolling(future_window)
        .max()
    )

    # ------------------------
    # 3) dropna and return
    # ------------------------
    model_df = out[["date"] + feat_cols + ["target_sep"]].dropna().reset_index(drop=True)

    X = model_df[feat_cols].values
    y = model_df["target_sep"].values

    if use_log_target:
        y = np.log10(y + 1.0)

    return model_df, feat_cols, X, y




# -----------------------------
# 5) 모델 학습/평가 (시간 순서 유지)
# -----------------------------
def train_and_evaluate(X, y):
    # 가장 단순한 baseline: RandomForestRegressor
    model = RandomForestRegressor(
        n_estimators=500,
        max_depth=8,
        random_state=0,
        n_jobs=-1
    )

    # 마지막 20%를 테스트로 (time split)
    n = len(X)
    split = int(n * 0.8)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, pred)

    return model, {"rmse": rmse, "r2": r2}


# -----------------------------
# 실행 예시
# -----------------------------
if __name__ == "__main__":
    year = 2002
    cme_dir = f"./{year}cme"          # 너 폴더 구조에 맞게
    dpd_path = f"./{year}_DPD.txt"    # 너 파일명에 맞게

    k_lag = 3  # 과거 3일 CME로 오늘 SEP 예측

    cme_daily = load_and_preprocess_cme_multifeature(year, cme_dir)
    hp_daily  = load_and_preprocess_hproton(dpd_path)

    merged = align_daily_series(cme_daily, hp_daily)

    ds, feature_cols, X, y = make_supervised_dataset(
        merged,
        k_lag=3,
        future_window=3,
        use_log_target=True
    )


    model, metrics = train_and_evaluate(X, y)

    print("Features:", feature_cols)
    print("Metrics:", metrics)
    print("Dataset length:", len(ds))


Features: ['cme_count_lag1', 'cme_count_lag2', 'cme_count_lag3', 'cme_count_rollsum3', 'cme_count_rollmax3', 'halo_count_lag1', 'halo_count_lag2', 'halo_count_lag3', 'halo_count_rollsum3', 'halo_count_rollmax3', 'partial_halo_count_lag1', 'partial_halo_count_lag2', 'partial_halo_count_lag3', 'partial_halo_count_rollsum3', 'partial_halo_count_rollmax3', 'width_max_lag1', 'width_max_lag2', 'width_max_lag3', 'width_max_rollsum3', 'width_max_rollmax3', 'width_mean_lag1', 'width_mean_lag2', 'width_mean_lag3', 'width_mean_rollsum3', 'width_mean_rollmax3', 'speed_linear_max_lag1', 'speed_linear_max_lag2', 'speed_linear_max_lag3', 'speed_linear_max_rollsum3', 'speed_linear_max_rollmax3', 'speed_linear_mean_lag1', 'speed_linear_mean_lag2', 'speed_linear_mean_lag3', 'speed_linear_mean_rollsum3', 'speed_linear_mean_rollmax3', 'speed_init_max_lag1', 'speed_init_max_lag2', 'speed_init_max_lag3', 'speed_init_max_rollsum3', 'speed_init_max_rollmax3', 'speed_final_max_lag1', 'speed_final_max_lag2', 's

In [2]:
print(
    "event days:",
    (ds["target_sep"] > 0).sum(),
    "/",
    len(ds)
)

event days: 361 / 361
